# Handling data

In this tutorial session we explore the fundamental features of [Pandas](http://pandas.pydata.org/) and we practice the basics of Data Wrangling.

----

*Credits to: [Chris Fonnesbeck](https://github.com/fonnesbeck)  and [Michele Catasta](https://github.com/pirroh)*

*Updated by: [Panayiotis Smeros](https://github.com/psmeros) and [Pablo Cañas](https://github.com/pcanas)*

# Introduction to Pandas

**pandas** is a Python package providing fast, flexible, and expressive data structures designed to work with *relational* or *labeled* data both. It is a fundamental high-level building block for doing practical, real world data analysis in Python. 

pandas is well suited for:

- Tabular data with heterogeneously-typed columns, as in an SQL table or Excel spreadsheet
- Ordered and unordered (not necessarily fixed-frequency) time series data.
- Arbitrary matrix data (homogeneously typed or heterogeneous) with row and column labels
- Any other form of observational / statistical data sets. The data actually need not be labeled at all to be placed into a pandas data structure


Key features:
    
- Easy handling of **missing data**
- **Size mutability**: columns can be inserted and deleted from DataFrame and higher dimensional objects
- Automatic and explicit **data alignment**: objects can be explicitly aligned to a set of labels, or the data can be aligned automatically
- Powerful, flexible **group by functionality** to perform split-apply-combine operations on data sets
- Intelligent label-based **slicing, fancy indexing, and subsetting** of large data sets
- Intuitive **merging and joining** data sets
- **Time series functionality**: date range generation and frequency conversion, moving window statistics, moving window linear regressions, date shifting and lagging, etc.

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.

If you have an error about missing modules, try installing them in the command line:
```
conda install pandas
conda install numpy
```

You will need to install the optional dependency `xlrd` to read Microsoft Excel's files. To do so, run:
```
conda install xlrd
```

## Pandas Data Structures

### Series

A **Series** is a single vector of data (like a NumPy 1-d array) with an *index* that labels each element in the vector.

In [2]:
counts = pd.Series([632, 1638, 569, 115])
counts

0     632
1    1638
2     569
3     115
dtype: int64

If an <b>index</b> is not specified, a default sequence of integers is assigned as the index. A NumPy array comprises the values of the `Series`, while the index is a pandas `Index` object.

In [3]:
counts.values

array([ 632, 1638,  569,  115])

In [4]:
counts.index

RangeIndex(start=0, stop=4, step=1)

We can assign meaningful labels to the index, if they are available:

In [5]:
bacteria = pd.Series([632, 1638, 569, 115], 
    index=['Firmicutes', 'Proteobacteria', 'Actinobacteria', 'Bacteroidetes'])

bacteria

Firmicutes         632
Proteobacteria    1638
Actinobacteria     569
Bacteroidetes      115
dtype: int64

These labels can be used to refer to the values in the `Series`.

In [6]:
bacteria['Actinobacteria'] #dictionary style

569

Notice that the indexing operation preserved the association between the values and the corresponding indices.

We can still use <b>positional indexing</b> if we wish.

In [7]:
bacteria[0]

632

NumPy's math functions and other operations can be applied to Series without losing the data structure.

In [8]:
bacteria + 3

Firmicutes         635
Proteobacteria    1641
Actinobacteria     572
Bacteroidetes      118
dtype: int64

We can also filter according to the values in the `Series`:

In [9]:
bacteria[bacteria>1000]

Proteobacteria    1638
dtype: int64

### Exercise
Filter the bacteria `Series` to keep only the values whose index end with the word "bacteria". Hint: have a look at the [endswith](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.endswith.html) function

In [10]:
bacteria[bacteria.index.str.endswith("bacteria")]

Proteobacteria    1638
Actinobacteria     569
dtype: int64

### DataFrame

Inevitably, we want to be able to store, view and manipulate data that is *multivariate*, where for every index there are multiple fields or columns of data, often of varying data type.

A `DataFrame` is a tabular data structure, encapsulating multiple series like columns in a spreadsheet. Data are stored internally as a 2-dimensional object, but the `DataFrame` allows us to represent and manipulate higher-dimensional data.

In [11]:
data = pd.DataFrame({'value':[632, 1638, 569, 115, 433, 1130, 754, 555],
                     'patient':[1, 1, 1, 1, 2, 2, 2, 2],
                     'phylum':['Firmicutes', 'Proteobacteria', 'Actinobacteria', 
    'Bacteroidetes', 'Firmicutes', 'Proteobacteria', 'Actinobacteria', 'Bacteroidetes']})
data

,value,patient,phylum
0,632,1,Firmicutes
1,1638,1,Proteobacteria
2,569,1,Actinobacteria
3,115,1,Bacteroidetes
4,433,2,Firmicutes
5,1130,2,Proteobacteria
6,754,2,Actinobacteria
7,555,2,Bacteroidetes


The `dtypes` attribute reveals the data type for each column in our DataFrame. 

- `int64` is numeric integer values 
- `object` strings (letters and numbers)
- `float64` floating-point values

In [12]:
data.dtypes

value       int64
patient     int64
phylum     object
dtype: object

If we wish to access columns, we can do so either by dict-like indexing or by attribute:

In [13]:
data['patient']

0    1
1    1
2    1
3    1
4    2
5    2
6    2
7    2
Name: patient, dtype: int64

In [14]:
data.value

0     632
1    1638
2     569
3     115
4     433
5    1130
6     754
7     555
Name: value, dtype: int64

If we want access to a row or column in a `DataFrame`, we index its `loc` attribute.

In [15]:
data.loc[3]

value                115
patient                1
phylum     Bacteroidetes
Name: 3, dtype: object

In [16]:
data.loc[3, 'patient']

1

In addition to using `loc` to select rows and columns by **label**, pandas also allows indexing by **position** using the `iloc` attribute.

So, we can query rows and columns by absolute position, rather than by name:

In [17]:
data.iloc[3] # in this case, index and absolute position match

value                115
patient                1
phylum     Bacteroidetes
Name: 3, dtype: object

In [18]:
data.iloc[3, 1] # 'patient' column

1

#### Exercise

Find the size of the dimension of `data`. Hint: the [shape](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shape.html) method might be useful

In [19]:
data.shape

(8, 3)

#### Exercise

Select the attribute `phylum` of the patient with index 4

In [20]:
data.iloc[4, 2] 

'Firmicutes'

We can create or modify columns by assignment:

In [21]:
data.value[[3,4,6]] = [14, 21, 5]
data

,value,patient,phylum
0,632,1,Firmicutes
1,1638,1,Proteobacteria
2,569,1,Actinobacteria
3,14,1,Bacteroidetes
4,21,2,Firmicutes
5,1130,2,Proteobacteria
6,5,2,Actinobacteria
7,555,2,Bacteroidetes


In [22]:
data['year'] = 2013
data

,value,patient,phylum,year
0,632,1,Firmicutes,2013
1,1638,1,Proteobacteria,2013
2,569,1,Actinobacteria,2013
3,14,1,Bacteroidetes,2013
4,21,2,Firmicutes,2013
5,1130,2,Proteobacteria,2013
6,5,2,Actinobacteria,2013
7,555,2,Bacteroidetes,2013


#### Exercise

From the `data` table above, return all rows for which the the value is greater than 1000.

In [23]:
data[data['value'] >1000]

,value,patient,phylum,year
1,1638,1,Proteobacteria,2013
5,1130,2,Proteobacteria,2013


#### Exercise

From the `data` table above, create a new column `value_group` whose value is 1 when `value<=1000` otherwise is 2.

In [24]:
data['value_group'] = 1
data.loc[data['value']>1000, 'value_group'] = 2

data

,value,patient,phylum,year,value_group
0,632,1,Firmicutes,2013,1
1,1638,1,Proteobacteria,2013,2
2,569,1,Actinobacteria,2013,1
3,14,1,Bacteroidetes,2013,1
4,21,2,Firmicutes,2013,1
5,1130,2,Proteobacteria,2013,2
6,5,2,Actinobacteria,2013,1
7,555,2,Bacteroidetes,2013,1


Specifying a `Series` as a new column causes its values to be added according to the `DataFrame`'s index:

In [25]:
data['treatment'] = pd.Series([0]*4 + [1]*2)
data

,value,patient,phylum,year,value_group,treatment
0,632,1,Firmicutes,2013,1,0.0
1,1638,1,Proteobacteria,2013,2,0.0
2,569,1,Actinobacteria,2013,1,0.0
3,14,1,Bacteroidetes,2013,1,0.0
4,21,2,Firmicutes,2013,1,1.0
5,1130,2,Proteobacteria,2013,2,1.0
6,5,2,Actinobacteria,2013,1,NaN
7,555,2,Bacteroidetes,2013,1,NaN


We can use the `drop` method to remove rows or columns, which by <b>default drops rows</b>. We can be explicit by using the `axis` argument:

In [26]:
data_notreatment = data.drop('treatment', axis=1)
data_notreatment

,value,patient,phylum,year,value_group
0,632,1,Firmicutes,2013,1
1,1638,1,Proteobacteria,2013,2
2,569,1,Actinobacteria,2013,1
3,14,1,Bacteroidetes,2013,1
4,21,2,Firmicutes,2013,1
5,1130,2,Proteobacteria,2013,2
6,5,2,Actinobacteria,2013,1
7,555,2,Bacteroidetes,2013,1


In [27]:
data_notreatment.drop(0)

,value,patient,phylum,year,value_group
1,1638,1,Proteobacteria,2013,2
2,569,1,Actinobacteria,2013,1
3,14,1,Bacteroidetes,2013,1
4,21,2,Firmicutes,2013,1
5,1130,2,Proteobacteria,2013,2
6,5,2,Actinobacteria,2013,1
7,555,2,Bacteroidetes,2013,1


## Importing data

A key, but often under-appreciated, step in data analysis is importing the data that we wish to analyze. Though it is easy to load basic data structures into Python using built-in tools or those provided by packages like NumPy, it is non-trivial to import structured data well, and to easily convert this input into a robust data structure:

    genes = np.loadtxt("genes.csv", delimiter=",", dtype=[('gene', '|S10'), ('value', '<f4')])

Pandas provides a convenient set of functions for importing tabular data from a number of formats directly into a `DataFrame` object. These functions include a slew of options to perform type inference, indexing, parsing, iterating and cleaning automatically as data are imported.

Let's start with some more bacteria data, stored in csv format. This table can be read into a DataFrame using `read_csv`:

In [28]:
mb = pd.read_csv("./data/microbiome.csv", sep=',')
mb.head()

,Taxon,Patient,Group,Tissue,Stool
0,Firmicutes,1,0,136,4182
1,Firmicutes,2,1,1174,703
2,Firmicutes,3,0,408,3946
3,Firmicutes,4,1,831,8605
4,Firmicutes,5,0,693,50


Notice that `read_csv` automatically considered the first row in the file to be a header row.

#### Missing Values

Most real-world data is incomplete, with values missing due to incomplete observation, data entry or transcription error, or other reasons. Pandas will automatically recognize and parse common missing data indicators, including `NA` and `NULL`.

In [29]:
pd.read_csv("./data/microbiome_missing.csv").head(10)

,Taxon,Patient,Tissue,Stool
0,Firmicutes,1,632,305.0
1,Firmicutes,2,136,4182.0
2,Firmicutes,3,NaN,703.0
3,Firmicutes,4,408,3946.0
4,Firmicutes,5,831,8605.0
5,Firmicutes,6,693,50.0
6,Firmicutes,7,718,717.0
7,Firmicutes,8,173,33.0
8,Firmicutes,9,228,NaN
9,Firmicutes,10,162,3196.0


Above, Pandas recognized `NA` and an empty field as missing data.

In [30]:
pd.isnull(pd.read_csv("./data/microbiome_missing.csv")).head(10)

,Taxon,Patient,Tissue,Stool
0,False,False,False,False
1,False,False,False,False
2,False,False,True,False
3,False,False,False,False
4,False,False,False,False
5,False,False,False,False
6,False,False,False,False
7,False,False,False,False
8,False,False,False,True
9,False,False,False,False


#### Exercise

You can use the [isin](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isin.html) method query a DataFrame based upon a list of values as follows: 

    data['phylum'].isin(['Firmicutes', 'Bacteroidetes'])

Use `isin` on `baseball`to find all players that played for the Los Angeles Dodgers (LAN) or the San Francisco Giants (SFN). How many records contain these values?

In [31]:
baseball = pd.read_csv("./data/baseball.csv", index_col='id')

In [32]:
len(baseball[baseball.team.isin(['LAN', 'SFN'])])

15

## Operations

`DataFrame` and `Series` objects allow for several operations to take place either on a single object, or between two or more objects.

For example, we can perform arithmetic on the elements of two objects, such as combining baseball statistics across years. First, let's (artificially) construct two Series, consisting of home runs hit in years 2006 and 2007, respectively:

In [33]:
hr2006 = baseball.loc[baseball.year==2006, 'hr']
hr2006.index = baseball.player[baseball.year==2006]

hr2007 = baseball.loc[baseball.year==2007, 'hr']
hr2007.index = baseball.player[baseball.year==2007]

In [34]:
print(hr2006.head(10))
print(hr2007.head(10))

player
womacto01     1
schilcu01     0
myersmi01     0
helliri01     0
johnsra05     0
finlest01     6
gonzalu01    15
seleaa01      0
Name: hr, dtype: int64
player
francju01     0
francju01     1
zaungr01     10
witasja01     0
williwo02     1
wickmbo01     0
wickmbo01     0
whitero02     4
whiteri01     0
wellsda01     0
Name: hr, dtype: int64


Now, let's add them together, in hopes of getting 2-year home run totals:

In [35]:
hr_total = hr2006 + hr2007
hr_total.head(20)

player
alomasa02    NaN
aloumo01     NaN
ausmubr01    NaN
benitar01    NaN
benitar01    NaN
biggicr01    NaN
bondsba01    NaN
cirilje01    NaN
cirilje01    NaN
claytro01    NaN
claytro01    NaN
clemero02    NaN
coninje01    NaN
coninje01    NaN
cormirh01    NaN
delgaca01    NaN
easleda01    NaN
edmonji01    NaN
embreal01    NaN
finlest01    7.0
Name: hr, dtype: float64

Pandas' data alignment places `NaN` values for labels that do not overlap in the two Series. In fact, there are only 6 players that occur in both years.

In [36]:
hr_total[hr_total.notnull()]

player
finlest01     7.0
gonzalu01    30.0
johnsra05     0.0
myersmi01     0.0
schilcu01     0.0
seleaa01      0.0
Name: hr, dtype: float64

While we do want the operation to honor the data labels in this way, we probably do not want the missing values to be filled with `NaN`. We can use the `add` method to calculate player home run totals by using the `fill_value` argument to insert a zero for home runs where labels do not overlap:

In [37]:
hr2007.add(hr2006, fill_value=0).head(20)

player
alomasa02     0.0
aloumo01     13.0
ausmubr01     3.0
benitar01     0.0
benitar01     0.0
biggicr01    10.0
bondsba01    28.0
cirilje01     0.0
cirilje01     2.0
claytro01     0.0
claytro01     1.0
clemero02     0.0
coninje01     0.0
coninje01     6.0
cormirh01     0.0
delgaca01    24.0
easleda01    10.0
edmonji01    12.0
embreal01     0.0
finlest01     7.0
Name: hr, dtype: float64

### Exercise

Find the player with the most home runs of season 2006

In [38]:
hr2006[hr2006 == hr2006.max()]

player
gonzalu01    15
Name: hr, dtype: int64

## Missing data

The occurence of missing data is so prevalent that it pays to use tools like Pandas, which seamlessly integrates missing data handling so that it can be dealt with easily, and in the manner required by the analysis at hand.

Missing data are represented in `Series` and `DataFrame` objects by the `NaN` floating point value. However, `None` is also treated as missing, since it is commonly used as such in other contexts (*e.g.* NumPy).

In [39]:
foo = pd.Series([np.nan, -3, None, 'foobar'])
foo

0       NaN
1        -3
2      None
3    foobar
dtype: object

In [40]:
foo.isnull()

0     True
1    False
2     True
3    False
dtype: bool

Missing values may be dropped using `dropna`

In [41]:
foo.dropna()

1        -3
3    foobar
dtype: object

Rather than omitting missing data from an analysis, in some cases it may be suitable to fill the missing value in, either with a default value (such as zero) or a value that is either imputed or carried forward/backward from similar data points. We can do this programmatically in Pandas with the `fillna` argument.

In [42]:
bacteria_dict = { 'Firmicutes': 632, 
                  'Proteobacteria': 1638, 
                  'Actinobacteria': 569,
                  'Bacteroidetes': 115 }
bacteria2 = pd.Series(bacteria_dict, 
                      index=['Cyanobacteria','Firmicutes',
                             'Proteobacteria','Actinobacteria'])
bacteria2

Cyanobacteria        NaN
Firmicutes         632.0
Proteobacteria    1638.0
Actinobacteria     569.0
dtype: float64

In [43]:
bacteria2.fillna(0)

Cyanobacteria        0.0
Firmicutes         632.0
Proteobacteria    1638.0
Actinobacteria     569.0
dtype: float64

Missing values can also be interpolated, using any one of a variety of methods:

In [44]:
bacteria2.fillna(method='bfill')

Cyanobacteria      632.0
Firmicutes         632.0
Proteobacteria    1638.0
Actinobacteria     569.0
dtype: float64

## Data summarization

We often wish to summarize data in `Series` or `DataFrame` objects, so that they can more easily be understood or compared with similar data. The NumPy package contains several functions that are useful here, but several summarization or reduction methods are built into Pandas data structures.

In [45]:
baseball.sum()

player    womacto01schilcu01myersmi01helliri01johnsra05f...
year                                                 200692
stint                                                   113
team      CHNBOSNYAMILNYASFNARILANATLNYNTORTBAHOUARIATLM...
lg        NLALALNLALNLNLNLNLNLALALNLNLNLALNLNLNLNLALALNL...
g                                                      5238
ab                                                    13654
r                                                      1869
h                                                      3582
X2b                                                     739
X3b                                                      55
hr                                                      437
rbi                                                  1847.0
sb                                                    138.0
cs                                                     46.0
bb                                                     1549
so                                      

Clearly, `sum` is more meaningful for some columns than others. For methods like `mean` for which application to string variables is not just meaningless, but impossible, these columns are <b>automatically exculded</b>:

In [46]:
baseball.mean()

/var/folders/kh/cn1l2cp106942cgw5ltg5tz80000gn/T/ipykernel_77963/2942499688.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  baseball.mean()


year     2006.92
stint       1.13
g          52.38
ab        136.54
r          18.69
h          35.82
X2b         7.39
X3b         0.55
hr          4.37
rbi        18.47
sb          1.38
cs          0.46
bb         15.49
so         24.08
ibb         1.77
hbp         1.12
sh          1.38
sf          1.20
gidp        3.54
dtype: float64

The important difference between NumPy's functions and Pandas' methods is that the latter have built-in support for handling missing data.

In [47]:
bacteria2

Cyanobacteria        NaN
Firmicutes         632.0
Proteobacteria    1638.0
Actinobacteria     569.0
dtype: float64

In [48]:
bacteria2.mean()

946.3333333333334

A useful summarization that gives a quick snapshot of multiple statistics for a `Series` or `DataFrame` is `describe`:

In [49]:
baseball.describe()

,year,stint,g,ab,r,h,X2b,X3b,hr,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,gidp
count,100.00000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000
mean,2006.92000,1.130000,52.380000,136.540000,18.69000,35.820000,7.390000,0.550000,4.370000,18.47000,1.380000,0.460000,15.490000,24.080000,1.770000,1.12000,1.380000,1.200000,3.540000
std,0.27266,0.337998,48.031299,181.936853,27.77496,50.221807,11.117277,1.445124,7.975537,28.34793,3.694878,1.067613,25.812649,32.804496,5.042957,2.23055,2.919042,2.035046,5.201826
min,2006.00000,1.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,2007.00000,1.000000,9.500000,2.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000
50%,2007.00000,1.000000,33.000000,40.500000,2.00000,8.000000,1.000000,0.000000,0.000000,2.00000,0.000000,0.000000,1.000000,7.000000,0.000000,0.00000,0.000000,0.000000,1.000000
75%,2007.00000,1.000000,83.250000,243.750000,33.25000,62.750000,11.750000,1.000000,6.000000,27.00000,1.000000,0.000000,19.250000,37.250000,1.250000,1.00000,1.000000,2.000000,6.000000
max,2007.00000,2.000000,155.000000,586.000000,107.00000,159.000000,52.000000,12.000000,35.000000,96.00000,22.000000,6.000000,132.000000,134.000000,43.000000,11.00000,14.000000,9.000000,21.000000


`describe` can detect non-numeric data and sometimes yield useful information about it.

In [50]:
baseball.player.describe()

count           100
unique           82
top       wellsda01
freq              2
Name: player, dtype: object

### Exercise

Find the average number of home runs of season 2006

In [51]:
hr2006.mean()

2.75

# Data Wrangling with Pandas

Now that we have been exposed to the basic functionality of Pandas, let's explore some more advanced features that will be useful when addressing more complex data management tasks.

As most statisticians/data analysts will admit, often the lion's share of the time spent implementing an analysis is devoted to preparing the data itself, rather than to coding or running a particular model that uses the data. This is where Pandas and  Python's standard library are beneficial, providing high-level, flexible, and efficient tools for manipulating your data as needed.


## Merging and joining DataFrame objects

In Pandas, we can combine tables according to the value of one or more *keys* that are used to identify rows, much like an index. Using a trivial example:

In [52]:
df1 = pd.DataFrame(dict(id=range(4), age=np.random.randint(18, 31, size=4)))
df2 = pd.DataFrame(dict(id=list(range(3))+list(range(3)), 
                        score=np.random.random(size=6)))

df1

,id,age
0,0,19
1,1,28
2,2,18
3,3,19


In [53]:
df2

,id,score
0,0,0.949382
1,1,0.159807
2,2,0.653970
3,0,0.025132
4,1,0.662307
5,2,0.624174


In [54]:
pd.merge(df1, df2)

,id,age,score
0,0,19,0.949382
1,0,19,0.025132
2,1,28,0.159807
3,1,28,0.662307
4,2,18,0.653970
5,2,18,0.624174


Notice that without any information about which column to use as a key, Pandas did the right thing and used the `id` column in both tables. Unless specified otherwise, `merge` will used any common column names as keys for merging the tables. 

Notice also that `id=3` from `df1` was omitted from the merged table. This is because, by default, `merge` performs an **inner join** on the tables, meaning that the merged table represents an intersection of the two tables.

In [55]:
pd.merge(df1, df2, how='outer')

,id,age,score
0,0,19,0.949382
1,0,19,0.025132
2,1,28,0.159807
3,1,28,0.662307
4,2,18,0.653970
5,2,18,0.624174
6,3,19,NaN


The **outer join** above yields the union of the two tables, so all rows are represented, with missing values inserted as appropriate. One can also perform **right** and **left** joins to include all rows of the right or left table (*i.e.* first or second argument to `merge`), but not necessarily the other.

## Data aggregation and GroupBy operations

One of the most powerful features of Pandas is its [**GroupBy**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) functionality. On some occasions we may want to perform operations on *groups* of observations within a dataset. For example:

* **aggregation**, such as computing the sum of mean of each group, which involves applying a function to each group and returning the aggregated results
* **slicing** the DataFrame into groups and then doing something with the resulting slices (*e.g.* plotting)
* group-wise **transformation**, such as standardization/normalization

In [56]:
cdystonia = pd.read_csv("./data/cdystonia.csv", index_col=None)
cdystonia.head()

,patient,obs,week,site,id,treat,age,sex,twstrs
0,1,1,0,1,1,5000U,65,F,32
1,1,2,2,1,1,5000U,65,F,30
2,1,3,4,1,1,5000U,65,F,24
3,1,4,8,1,1,5000U,65,F,37
4,1,5,12,1,1,5000U,65,F,39


In [57]:
cdystonia_grouped = cdystonia.groupby(cdystonia.patient)

However, the grouping is only an intermediate step; for example, we may want to **iterate** over each of the patient groups:

A common data analysis procedure is the **split-apply-combine** operation, which groups subsets of data together, applies a function to each of the groups, then recombines them into a new data table.

For example, we may want to aggregate our data with with some function.

<img src="./img/split-apply-combine.jpg">

For example, we can compute the mean of each attribute per patient group

In [58]:
cdystonia_grouped.mean().head() #equivalent 

,obs,week,site,id,age,twstrs
patient,,,,,,
1,3.5,7.0,1.0,1.0,65.0,33.000000
2,3.5,7.0,1.0,2.0,70.0,47.666667
3,3.5,7.0,1.0,3.0,64.0,30.500000
4,2.5,3.5,1.0,4.0,59.0,60.000000
5,3.5,7.0,1.0,5.0,76.0,46.166667


Notice that the `treat` and `sex` variables are <b>not included in the aggregation</b>. Since it does not make sense to aggregate string variables, these columns are simply ignored by the method.

If we wish, we can easily aggregate according to multiple keys:

In [59]:
cdystonia.groupby(['week','site']).mean().head()

patient  obs   id        age     twstrs
week site                                         
0    1         6.5  1.0  6.5  59.000000  43.083333
     2        19.5  1.0  7.5  53.928571  51.857143
     3        32.5  1.0  6.5  51.500000  38.750000
     4        42.5  1.0  4.5  59.250000  48.125000
     5        49.5  1.0  3.5  51.833333  49.333333

### Exercise
Using `cdystonia_grouped` variable from above, calculate the median of the column `twstrs` for each patient group

In [60]:
sol = cdystonia_grouped['twstrs'].quantile(0.5)

sol.head(10)

patient
1     34.0
2     50.5
3     30.5
4     61.5
5     48.5
6     48.0
7     42.0
8     32.5
9     35.5
10    20.5
Name: twstrs, dtype: float64

## References

[1] [Python for Data Analysis](http://shop.oreilly.com/product/0636920023784.do) Wes McKinney